<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-validation-prediction-files" data-toc-modified-id="Generate-validation-prediction-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate validation prediction files</a></span></li><li><span><a href="#create-metrics-from-validation-files" data-toc-modified-id="create-metrics-from-validation-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>create metrics from validation files</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import pdb
from argparse import ArgumentParser
import shlex
from tqdm import tqdm

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from pMHC import OUTPUT_FOLDER, SEP, \
    SPLITS, SPLIT_TRAIN, SPLIT_VAL, SPLIT_TEST, \
    SPLIT_VAL_PROTEINS, SPLIT_TEST_PROTEINS, SPLIT_VAL_MHC_ALLELES, SPLIT_TEST_MHC_ALLELES, \
    VIEWS, VIEW_SA, VIEW_SAMA, VIEW_DECONV, \
    INPUT_PEPTIDE, INPUT_CONTEXT, \
    BENCHMARK_SA, BENCHMARK_MA, \
    VALIDATION_PERFORMANCE_FILENAME, TEST_PERFORMANCE_FILENAME
from pMHC.logic import PresentationPredictor
from pMHC.data import from_data, to_input
from pMHC.data.example import Sample, Peptide, Observation

tqdm.pandas()

In [2]:
import torch

from pytorch_lightning.utilities.seed import seed_everything

import pMHC
from pMHC import POSITIVE_THRESHOLD
from pMHC.logic.utils import load_latest, list_versions, validate_checkpoints, predict_checkpoints, evaluate_checkpoints
from pMHC.data import from_input
from pMHC.data.utils import get_input_rep_PSEUDO, get_input_rep_FULL, convert_examples_to_batch
from pMHC.data.mhc_allele import MhcAllele
from pMHC.data.protein import Protein

In [3]:
from pMHC.data.split import load_split_mhc_alleles, load_split_proteins
from pMHC.data.view import create_views

In [4]:
pMHC.set_paths(r"/home/tux/Documents/MScProject")

Update project folder to: /home/tux/Documents/MScProject
Load permutation


In [5]:
proportion = 1.0

In [6]:
to_assess = [
    ("main", "CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001", "epoch=0-step=128494", proportion),
    ("main", "CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001", "epoch=1-step=1008417", proportion),
    ("main", "CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001", "epoch=2-step=1888340", proportion),
    ("main", "CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001", "epoch=3-step=2768263", proportion),
    ("main", "CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001", "epoch=4-step=3648186", proportion),
]

# Generate validation prediction files

In [7]:
predict_checkpoints(to_assess, SPLIT_VAL, VIEW_SAMA)

Validate: /home/tux/Documents/MScProject/output/main/CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001/checkpoints/epoch=0-step=128494


Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Global seed set to 42


PresentationPredictor.setup: 2021-07-30 15:05:20
MhcAllele.from_input


MhcAlleles from input: 11074it [00:00, 18407.37it/s]


Protein.from_input


...

   val: tp... 66583, tn... 8670838, fp... 41193, fn... 21418
   val: tp... 66791, tn... 8702368, fp... 41343, fn... 21530
   val: tp... 67019, tn... 8733911, fp... 41480, fn... 21622
   val: tp... 67249, tn... 8765446, fp... 41625, fn... 21712
   val: tp... 67480, tn... 8796974, fp... 41777, fn... 21801
   val: tp... 67710, tn... 8828494, fp... 41937, fn... 21891
   val: tp... 67950, tn... 8860042, fp... 42069, fn... 21971
   val: tp... 68156, tn... 8891581, fp... 42210, fn... 22085
   val: tp... 68384, tn... 8923117, fp... 42354, fn... 22177
   val: tp... 68661, tn... 8954645, fp... 42506, fn... 22220
   val: tp... 68934, tn... 8986185, fp... 42646, fn... 22267
   val: tp... 69189, tn... 9017721, fp... 42790, fn... 22332
   val: tp... 69447, tn... 9049241, fp... 42950, fn... 22394
   val: tp... 69689, tn... 9080735, fp... 43136, fn... 22472
   val: tp... 69908, tn... 9112274, fp... 43277, fn... 22573
   val: tp... 70145, tn... 9143816, fp... 43415, fn... 22656
   val: tp... 70393, tn.

   val: tp... 99020, tn... 12864317, fp... 61154, fn... 31541
   val: tp... 99261, tn... 12895836, fp... 61315, fn... 31620
   val: tp... 99516, tn... 12927366, fp... 61465, fn... 31685
   val: tp... 99758, tn... 12958877, fp... 61634, fn... 31763
   val: tp... 100013, tn... 12990414, fp... 61777, fn... 31828
   val: tp... 100251, tn... 13021939, fp... 61932, fn... 31910
   val: tp... 100497, tn... 13053466, fp... 62085, fn... 31984
   val: tp... 100769, tn... 13085002, fp... 62229, fn... 32032
   val: tp... 101000, tn... 13116555, fp... 62356, fn... 32121
   val: tp... 101254, tn... 13148089, fp... 62502, fn... 32187
   val: tp... 101496, tn... 13179613, fp... 62658, fn... 32265
   val: tp... 101715, tn... 13211128, fp... 62823, fn... 32366
   val: tp... 101929, tn... 13242672, fp... 62959, fn... 32472
   val: tp... 102193, tn... 13274215, fp... 63096, fn... 32528
   val: tp... 102434, tn... 13305728, fp... 63263, fn... 32607
   val: tp... 102674, tn... 13337261, fp... 63410, fn... 32

   val: tp... 128356, tn... 16995401, fp... 80150, fn... 44125
   val: tp... 128587, tn... 17026936, fp... 80295, fn... 44214
   val: tp... 128798, tn... 17058470, fp... 80441, fn... 44323
   val: tp... 128996, tn... 17089998, fp... 80593, fn... 44445
   val: tp... 129164, tn... 17121553, fp... 80718, fn... 44597
   val: tp... 129375, tn... 17153091, fp... 80860, fn... 44706
   val: tp... 129588, tn... 17184623, fp... 81008, fn... 44813
   val: tp... 129781, tn... 17216179, fp... 81132, fn... 44940
   val: tp... 129971, tn... 17247717, fp... 81274, fn... 45070
   val: tp... 130148, tn... 17279272, fp... 81399, fn... 45213
   val: tp... 130354, tn... 17310825, fp... 81526, fn... 45327
   val: tp... 130558, tn... 17342372, fp... 81659, fn... 45443
   val: tp... 130755, tn... 17373919, fp... 81792, fn... 45566
   val: tp... 130976, tn... 17405467, fp... 81924, fn... 45665
   val: tp... 131177, tn... 17437007, fp... 82064, fn... 45784
   val: tp... 131347, tn... 17468563, fp... 82188, fn..

   val: tp... 155357, tn... 21127282, fp... 98349, fn... 59044
   val: tp... 155562, tn... 21158812, fp... 98499, fn... 59159
   val: tp... 155779, tn... 21190350, fp... 98641, fn... 59262
   val: tp... 155981, tn... 21221898, fp... 98773, fn... 59380
   val: tp... 156208, tn... 21253446, fp... 98905, fn... 59473
   val: tp... 156418, tn... 21284964, fp... 99067, fn... 59583
   val: tp... 156626, tn... 21316516, fp... 99195, fn... 59695
   val: tp... 156825, tn... 21348032, fp... 99359, fn... 59816
   val: tp... 157053, tn... 21379573, fp... 99498, fn... 59908
   val: tp... 157228, tn... 21411128, fp... 99623, fn... 60053
   val: tp... 157433, tn... 21442652, fp... 99779, fn... 60168
   val: tp... 157704, tn... 21474201, fp... 99910, fn... 60217
   val: tp... 157958, tn... 21505713, fp... 100078, fn... 60283
   val: tp... 158192, tn... 21537236, fp... 100235, fn... 60369
   val: tp... 158398, tn... 21568774, fp... 100377, fn... 60483
   val: tp... 158628, tn... 21600306, fp... 100525, 

   val: tp... 189782, tn... 25194010, fp... 118341, fn... 65899
   val: tp... 190078, tn... 25225532, fp... 118499, fn... 65923
   val: tp... 190377, tn... 25257078, fp... 118633, fn... 65944
   val: tp... 190646, tn... 25288601, fp... 118790, fn... 65995
   val: tp... 190859, tn... 25320097, fp... 118974, fn... 66102
   val: tp... 191160, tn... 25351632, fp... 119119, fn... 66121
   val: tp... 191403, tn... 25383153, fp... 119278, fn... 66198
   val: tp... 191664, tn... 25414678, fp... 119433, fn... 66257
   val: tp... 191926, tn... 25446230, fp... 119561, fn... 66315
   val: tp... 192177, tn... 25477740, fp... 119731, fn... 66384
   val: tp... 192407, tn... 25509272, fp... 119879, fn... 66474
   val: tp... 192691, tn... 25540819, fp... 120012, fn... 66510
   val: tp... 192994, tn... 25572351, fp... 120160, fn... 66527
   val: tp... 193308, tn... 25603905, fp... 120286, fn... 66533
   val: tp... 193557, tn... 25635442, fp... 120429, fn... 66604
   val: tp... 193792, tn... 25666968, fp

# create metrics from validation files

In [7]:
pMHC.data.setup_system(proportion, 99)

MhcAllele.from_input


MhcAlleles from input: 11074it [00:00, 17787.16it/s]


Protein.from_input


Proteins from input: 185930it [00:09, 19467.50it/s]


Sample.from_input


...

Peptide.from_input


Peptides from input: 429339it [00:22, 18746.91it/s]


Observation.from_input


Observations from input: 1959736it [01:41, 19375.21it/s]
100%|██████████| 1959736/1959736 [00:06<00:00, 301353.46it/s]


Decoy.to_input
Load decoys for 1959737 observations
decoys_1200000_1250000.csv
decoys_650000_700000.csv
decoys_1650000_1700000.csv
decoys_1000000_1050000.csv
decoys_1950000_2000000.csv
decoys_1150000_1200000.csv
decoys_1800000_1850000.csv
decoys_1450000_1500000.csv
decoys_950000_1000000.csv
decoys_850000_900000.csv
decoys_50000_100000.csv
decoys_1050000_1100000.csv
decoys_1300000_1350000.csv
decoys_1900000_1950000.csv
decoys_900000_950000.csv
decoys_700000_750000.csv
decoys_450000_500000.csv
decoys_200000_250000.csv
decoys_800000_850000.csv
decoys_500000_550000.csv
decoys_1750000_1800000.csv
decoys_1550000_1600000.csv
decoys_1600000_1650000.csv
decoys_300000_350000.csv
decoys_1850000_1900000.csv
decoys_1350000_1400000.csv
decoys_1500000_1550000.csv
decoys_1250000_1300000.csv
decoys_550000_600000.csv
decoys_1100000_1150000.csv
decoys_0_50000.csv
decoys_750000_800000.csv
decoys_350000_400000.csv
decoys_400000_450000.csv
decoys_1700000_1750000.csv
decoys_1400000_1450000.csv
decoys_100000_

In [8]:
info = evaluate_checkpoints(to_assess, [SPLIT_VAL_PROTEINS, SPLIT_VAL_MHC_ALLELES], [VIEW_SAMA])

Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=0-step=128494_1.0 - val-prot - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=0-step=128494_1.0 - val-mhc - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=1-step=1008417_1.0 - val-prot - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=1-step=1008417_1.0 - val-mhc - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=2-step=1888340_1.0 - val-prot - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=2-step=1888340_1.0 - val-mhc - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=3-step=2768263_1.0 - val-prot - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=3-step=2768263_1.0 - val-mhc - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=4-step=3648186_1.0 - val-prot - SAMA
Load: main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=4-step=3648186_1.0 - val-mhc - SAMA


In [9]:
list(info.keys())

['main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=0-step=128494_1.0',
 'main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=1-step=1008417_1.0',
 'main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=2-step=1888340_1.0',
 'main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=3-step=2768263_1.0',
 'main_CONTEXT-PSEUDO-HEAD_Cls-DECOY_19-LR_0.00001_epoch=4-step=3648186_1.0']

In [19]:
head = 'Cls'
decoys_per_obs = '19'
learning_rate = '0.00001'
view = 'SAMA'


for epoch, steps in zip([0, 1, 2, 3, 4], [128494, 1008417, 1888340, 2768263, 3648186]):
    model_name = f"main_CONTEXT-PSEUDO-HEAD_{head}-DECOY_{decoys_per_obs}-LR_{learning_rate}"
    checkpoint_name = f"{model_name}_epoch={epoch}-step={steps}_{proportion}"
    
    print(f"{head} {decoys_per_obs} {steps}", end="")
    #print(f"{info[checkpoint_name]}", end="")
    for split in ["val-mhc", "val-prot"]:
        for metric in ["AP", "roc_auc", "accuracy"]:
            print(f" & {info[checkpoint_name][split][view][metric]:.3f}", end="")

    print("")

Cls 19 128494 & 0.571 & 0.966 & 0.989 & 0.667 & 0.985 & 0.988
Cls 19 1008417 & 0.646 & 0.976 & 0.992 & 0.762 & 0.993 & 0.993
Cls 19 1888340 & 0.671 & 0.978 & 0.993 & 0.767 & 0.993 & 0.994
Cls 19 2768263 & 0.673 & 0.978 & 0.992 & 0.768 & 0.993 & 0.993
Cls 19 3648186 & 0.683 & 0.979 & 0.992 & 0.765 & 0.993 & 0.994
